# 12.0. Data Integration 

- Use '..\Pipeline\BeforeCollection\files.csv' : so hashes from URL Haus, Feodo Tracker and Threat Fox.
- Use '..\Pipeline\DataCollection_OTXAlienVault\hash_domain.csv' : (hash, domains) collected from OTXAlienVault
- Use '..\Pipeline\DataCollection_OTXAlienVault\hash_ips.csv' : (hash, ip address) collected from OTXAlienVault

to perform a look up on the Malware Bazaar in order to add an analysis 

In [1]:
from datetime import datetime
import os
import time
import pandas as pd 
import requests
import urllib3
import json
import sys
import numpy as np

pd.options.display.max_colwidth = 1000
pd.set_option('display.max_columns', None)

In [2]:
file = r'C:\Users\leona\OneDrive\Desktop\Tesi\Pipeline\Datasets\Pipeline\BeforeCollection\files.csv'

In [3]:
ipFiles = r'C:\Users\leona\OneDrive\Desktop\Tesi\Pipeline\Datasets\Pipeline\DataCollection_OTXAlienVault\hash_ips.csv'

In [5]:
urlFiles = r'C:\Users\leona\OneDrive\Desktop\Tesi\Pipeline\Datasets\Pipeline\DataCollection_OTXAlienVault\hash_domain.csv'

In [6]:
df_files = pd.read_csv(file, low_memory = False)
df_files

,Unnamed: 0,ioc
0,7123,7ca82baa3e7eddce41a25bf071216d1b
1,7125,91a2d790366e433f18036a85d935d7e628c652c35b00427f6ef809fa9ec1f69e
2,7130,c5521f937b54a93aaafd7ee393c8c598
3,7132,18b60825348e063c719f443680ba8425dc968ad9cfb4f07f91fea89e2b2d1f0a
4,7133,8b8d7db29e70be64c18a1cc8d02da680
5,7135,be6716415c8f117ac0d290833825ac7ddab7f18666cd6e68218db31f3da357c2
6,7138,91e5c5dbb6e64f5399cd4786f2e91192525b6582a088a8b583a7599a82838567
7,7139,e9476ead29f19bde9b3d61f166e63aef
8,7473,fd67ad03cc71d3397f962896365ed510
9,8250,b556bd47157695e3e0b279d56401026f


In [8]:
df_ip_files = pd.read_csv(ipFiles, low_memory = False)
df_ip_files

,Unnamed: 0,hash,detections.avast,detections.avg,detections.clamav,detections.msdefender,ip address,date
0,0,c13cce2e85db84a55e608527add7a9dc389694abd984cced400ee3d12b34b346,Win32:PWSX-gen\ [Trj],NaN,Win.Malware.Bulz-9880537-0,NaN,176.186.13.236,2022-04-27 12:08:47
1,1,1a213a9f2bb0984f35bbf43ebc3e34a54fe1347b3130ec51bfd75bc29f93786d,Win32:Malware-gen,NaN,Win.Ransomware.WannaCry-6313787-0,Ransom:Win32/CVE-2017-0147.A,223.8.200.190,2019-07-12 10:55:11
2,2,a2a0a4b91cbcf08126c70586521d2e7a0d6cf744058c314ffced7bdce97a40a1,Win32:PWSX-gen\ [Trj],NaN,NaN,NaN,37.0.14.214,2022-09-20 17:56:10
3,3,eaa2a9653157f3e52a379616fcde0911decaf0f069d3ee3f6b31f2d6087afe58,NaN,NaN,NaN,#LowFiCheckAVFolders,178.236.247.102,2023-09-15 17:41:53
4,4,d30d0f7ce6e737e09c188b5a3033f94876e9175417b002be02579fe83065ecb5,NaN,NaN,Win.Malware.Uztuby-9848412-0,Backdoor:MSIL/Quasar.GG!MTB,74.141.196.43,2023-10-11 02:39:52
...,...,...,...,...,...,...,...,...
1114,1114,2f411aee32eaff160d1be7cf3d8fbcd213770704172dca42d2e27b97bcbe16ca,NaN,NaN,NaN,NaN,41.86.5.198,2023-09-24 08:18:33
1115,1115,e364b3a5d68061977e17b548789190a4909ca5fbe87ca32a46beefcce367e3bb,Win32:Trojan-gen,NaN,Win.Malware.Ursu-9854581-0,ALF:JASYP:TrojanWin32/Ymacco!atmnm,185.203.116.147,2021-08-13 09:06:20
1116,1116,5189e968eb81dfb742d6e845b59cb20ec3007d12436fb5dabdbda78e1e49b598,NaN,NaN,Win.Trojan.Darkkomet-6745294-0,VirTool:Win32/CeeInject.AJJ!bit,3.133.207.110,2023-09-15 09:36:07
1117,1117,f50687084b8f8d0fc7a833a7eea363e1acf4f1812c5e9cb1e1d4293a519fdc39,Win32:Trojan-gen,NaN,NaN,NaN,80.66.75.34,2023-01-27 09:31:22


In [9]:
df_url_files = pd.read_csv(urlFiles, low_memory = False)
df_url_files

,Unnamed: 0,hash,detections.avast,detections.avg,detections.clamav,detections.msdefender,domain,date
0,0,0466d098c4beb3177c544801235119985f13eceebaa61405088f6fcd24fd6e96,NaN,NaN,Win.Trojan.Redline-9938775-1,NaN,gr.treef.top,2023-08-17 17:38:03
1,1,93a6da3e99aa31bef56be969ef66df9ae5aa2a3f7c744fa977486d78384c4de9,NaN,NaN,Win.Trojan.Redline-9938775-1,NaN,no.pleaz.top,2023-08-11 09:14:03
2,2,3d5aa422a120da634ebad364a3af6cee0be02a23210cec73bf692bbee545d472,NaN,NaN,NaN,ALF:HeraklezEval:Trojan:MSIL/AgentTesla.AR!rfn,thecookieisthere.duckdns.org,2023-09-16 18:11:42
3,3,53efa0ea3650452f5808cd2980b3ea7e0e3038ea283905355b406de8db6844de,NaN,NaN,Win.Trojan.NanoCore-9852758-0,Backdoor:MSIL/Nanocore.S!MTB,xdanetnow.duckdns.org,2023-09-26 02:17:14
4,4,659e11b099e01bd11e0e708e2e6cb9faf762eb1efd4b343b48f41a126d618f6a,NaN,NaN,Win.Trojan.Agent-6509946-0,NaN,silentlegion.duckdns.org,2023-09-20 12:54:18
...,...,...,...,...,...,...,...,...
1767,1767,87b05c7108251ef607d1b9bd1d717824c6a93d608e58d8e9553e403fa81e5ed9,Win32:PWSX-gen\ [Trj],NaN,NaN,NaN,presh147osidufhj.ddns.net,2022-06-09 19:21:12
1768,1768,a6b33cbfee592d34c4773d04ff1d3a06bc99d192db31bd1074a81ce25bdd049e,NaN,NaN,NaN,NaN,pkvithtosh17.com,2022-10-12 10:10:46
1769,1769,a6b33cbfee592d34c4773d04ff1d3a06bc99d192db31bd1074a81ce25bdd049e,NaN,NaN,NaN,NaN,pkvithtosh11.com,2022-10-12 10:10:46
1770,1770,d6aaa66ca79bc49452a184140f4f08c324f84f38a3087dac3e44b67d426c15a9,NaN,NaN,NaN,ALF:HeraklezEval:HackTool:Win32/DefenderControl,ekb.tuktuk.ug,2023-05-27 19:38:41


In [11]:
# Join Hashes in a list

In [12]:
lista_hash =  list(set(list(set(df_url_files['hash'])) + list(set(df_ip_files['hash'])) + list(set(df_files['ioc']))))
lista_hash

['58edde581177c6f96caa4da4cd73396afee160b9152692a664b28aa67dd0d100',
 '3b96a384f9989f28315eb374eae991e70ed07b88665cbf831f24bd574502feac',
 '3363270a2950c8518e7883ea147b3f947b3a96aa31d37a990c73f975946335ce',
 'cf1260b1d0686dbee4a5c9e3bcd09eff07bfae0a53d17356b72150f59ad1a497',
 '28b6bbc6b1d6ee21ccdc25c5eefccad5e21556954542cd19a0a87f7dde91df78',
 '461767cc1783f4ee7ec72bd279fec8c147c6cbff73e65bfb8766ef6ca05cc56f',
 '430d5d4db81a8e2e4cdcc7c572d3aefc7ced09a2a77246fc28297271ee646f4b',
 'f8c5b292bc728eb416c72e1e651be6ed8690a887512f61b7f2b29928b7d38103',
 '40175d0d99f5997724d290c09d0bc80c3f2e638ba8679b77af677c730f5869cd',
 'f47268ae527131b29e516b88aa7848b06aba712ca65021637eaffdb341064de7',
 '136f493c2e33a73d1479b33857c69a044ab69eb2b665f61c1d9f1a86e97750ce',
 '7ce982241ab41c3f0747929ed5b117020992dcaf689de9fb6464191a8e7658b8',
 '52ce8febd88643e68edd8e379b798bac51ce15c7b8b5932a76cec009a8a5ea35',
 'aad93ff025a725de6d3746c2e98126105b7a7f126b7340c540e13fa861c9e268',
 '4f6e4c0b82959a5a48456f137bbbca8c

In [13]:
len(lista_hash)

2659

In [15]:
df = pd.DataFrame()
df['hash'] = lista_hash
df

,hash
0,58edde581177c6f96caa4da4cd73396afee160b9152692a664b28aa67dd0d100
1,3b96a384f9989f28315eb374eae991e70ed07b88665cbf831f24bd574502feac
2,3363270a2950c8518e7883ea147b3f947b3a96aa31d37a990c73f975946335ce
3,cf1260b1d0686dbee4a5c9e3bcd09eff07bfae0a53d17356b72150f59ad1a497
4,28b6bbc6b1d6ee21ccdc25c5eefccad5e21556954542cd19a0a87f7dde91df78
...,...
2654,5de4932cb0ccab3fec711135cdd1af5a09c26a870bfb2d2adc650d0fdbf16488
2655,f10dd5ce32d72489dd2696e5b58ee115f3cd065969a61a6613cab27bcd57de70
2656,f3fcbb0fedb1e3b732185aebbf845ca185c950ca3635026d8a754312220577c9
2657,dba8a4717d516772b8cc06bd258265f21f7db6ddab62328917524c5a243c67a1


In [16]:
df.to_csv(r'C:\Users\leona\OneDrive\Desktop\Tesi\Pipeline\Datasets\Pipeline\DataIntegration_OTXAlienVault\hash.csv')